In [3]:
!python --version

Python 3.8.12


# Sauvegarde du fichier data dans l'espace de stockage Azure

In [102]:
from azureml.core import Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath

ws = Workspace(subscription_id="d5bb9744-4790-446f-b7e1-591e22995cc7",
               resource_group="OpenClassrooms",
               workspace_name="OC_IA")

datastore = Datastore.get(ws, 'workspaceblobstore')
ds = Dataset.File.upload_directory(src_dir='./data/',
           target=DataPath(datastore,  'UI/data'),
           show_progress=True)

Validating arguments.
Arguments validated.
Uploading file to UI/data
Uploading an estimated of 3475 files
Uploading ./data/bremen_000256_000019_leftImg8bit.png
Uploaded ./data/bremen_000256_000019_leftImg8bit.png, 1 files out of an estimated total of 3475
Uploading ./data/frankfurt_000001_046504_leftImg8bit.png
Uploaded ./data/frankfurt_000001_046504_leftImg8bit.png, 2 files out of an estimated total of 3475
Uploading ./data/darmstadt_000022_000019_leftImg8bit.png
Uploaded ./data/darmstadt_000022_000019_leftImg8bit.png, 3 files out of an estimated total of 3475
Uploading ./data/strasbourg_000001_065214_leftImg8bit.png
Uploaded ./data/strasbourg_000001_065214_leftImg8bit.png, 4 files out of an estimated total of 3475
Uploading ./data/stuttgart_000082_000019_leftImg8bit.png
Uploaded ./data/stuttgart_000082_000019_leftImg8bit.png, 5 files out of an estimated total of 3475
Uploading ./data/hamburg_000000_061790_leftImg8bit.png
Uploaded ./data/hamburg_000000_061790_leftImg8bit.png, 6 files 

Uploaded ./data/frankfurt_000001_041354_leftImg8bit.png, 2102 files out of an estimated total of 3475
Uploading ./data/dusseldorf_000066_000019_leftImg8bit.png
Uploaded ./data/dusseldorf_000066_000019_leftImg8bit.png, 2103 files out of an estimated total of 3475
Uploading ./data/frankfurt_000001_058914_leftImg8bit.png
Uploaded ./data/frankfurt_000001_058914_leftImg8bit.png, 2104 files out of an estimated total of 3475
Uploading ./data/dusseldorf_000115_000019_leftImg8bit.png
Uploaded ./data/dusseldorf_000115_000019_leftImg8bit.png, 2105 files out of an estimated total of 3475
Uploading ./data/munster_000137_000019_leftImg8bit.png
Uploaded ./data/munster_000137_000019_leftImg8bit.png, 2106 files out of an estimated total of 3475
Uploading ./data/munster_000044_000019_leftImg8bit.png
Uploaded ./data/munster_000044_000019_leftImg8bit.png, 2107 files out of an estimated total of 3475
Uploading ./data/stuttgart_000151_000019_leftImg8bit.png
Uploaded ./data/stuttgart_000151_000019_leftImg8bi

Uploading ./data/zurich_000081_000019_leftImg8bit.png
Uploaded ./data/zurich_000081_000019_leftImg8bit.png, 3464 files out of an estimated total of 3475
Uploading ./data/hanover_000000_052512_leftImg8bit.png
Uploaded ./data/hanover_000000_052512_leftImg8bit.png, 3465 files out of an estimated total of 3475
Uploading ./data/erfurt_000053_000019_leftImg8bit.png
Uploaded ./data/erfurt_000053_000019_leftImg8bit.png, 3466 files out of an estimated total of 3475
Uploading ./data/lindau_000058_000019_leftImg8bit.png
Uploaded ./data/lindau_000058_000019_leftImg8bit.png, 3467 files out of an estimated total of 3475
Uploading ./data/frankfurt_000000_000294_leftImg8bit.png
Uploaded ./data/frankfurt_000000_000294_leftImg8bit.png, 3468 files out of an estimated total of 3475
Uploading ./data/jena_000013_000019_leftImg8bit.png
Uploaded ./data/jena_000013_000019_leftImg8bit.png, 3469 files out of an estimated total of 3475
Uploading ./data/aachen_000123_000019_leftImg8bit.png
Uploaded ./data/aachen_0

# Accès aux images stockées sur Azure

In [1]:
from azureml.core import Workspace, Dataset, Datastore

# Connection à l'espace de travail d'Azure
ws = Workspace(subscription_id="d5bb9744-4790-446f-b7e1-591e22995cc7",
               resource_group="OpenClassrooms",
               workspace_name="OC_IA")


# azureml-core version 1.0.72 ou supérieure obligatoire
datastore = Datastore.get(ws, 'workspaceblobstore')

def load_img_from_azure(name):
    return Dataset.File.from_files((datastore, f'UI/data/{name}leftImg8bit.png')).download()

# Sauvegarde d'un model sur Azure

In [1]:
import cv2
import os
import numpy as np
from functions import show_2img
from tensorflow.keras.models import load_model

    
list_dire = os.listdir('../label/')
name = [path.split('gtFine_labelIds.png')[0] for path in list_dire]

X = np.array([cv2.imread(f'data/{name[u]}leftImg8bit.png') for u in range(0,10)])
y_true = np.array([cv2.resize(cv2.imread(f'../label/{name[u]}gtFine_labelIds.png', 0), 
                              (1024, 512)) for u in range(0,10)])

y_pred = np.array([[i.argmax() for i in u] for u in model_vgg_unet.predict(X[:1])[0].reshape((512, 1024, 8))])
show_2img(y_true[0],y_pred)

ModuleNotFoundError: No module named 'cv2'

In [ ]:
from azureml.core.model import Model

model_vgg_unet = load_model('model_vgg_unet/')

# Sauvegarde des modèle dans l'espacce de travail d'Azure
model = Model.register(model_path='./model_vgg_unet',
                       model_name="Model_vgg_unet",
                       tags={'area': "CV", 'type': "CNN"},
                       description="First Model Unet",
                       workspace=ws)

In [ ]:
from azureml.core.model import Model
from azureml.core import Workspace
from keras.models import load_model

# Connection à l'espace de travail d'Azure
ws = Workspace(subscription_id="d5bb9744-4790-446f-b7e1-591e22995cc7",
               resource_group="OpenClassrooms",
               workspace_name="OC_IA")
try :
    Model(ws, 'Model_vgg_unet').download()
except:
    pass

# Déploiment d'un modèle d'inférence

In [ ]:
%%time
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice

# Création des ressource environements de déploiement
source_directory = "./dependances"

env_tf_sk = Environment('tensorflow-scikit')
env_tf_sk.python.conda_dependencies.add_pip_package("azureml-core")
env_tf_sk.python.conda_dependencies.add_pip_package("scikit-learn")
env_tf_sk.python.conda_dependencies.add_pip_package("tensorflow")
env_tf_sk.python.conda_dependencies.add_pip_package("Pillow")

inference_config = InferenceConfig(source_directory=source_directory,
                                   entry_script="y/score.py",
                                   environment=env_tf_sk)

# Configuration du déploiement
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 4,auth_enabled=True)

# Création d'une instance de déploiement
service = Model.deploy(
    workspace = ws,
    name = "modelunet",
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config)

# Déploiement du modèle
service.wait_for_deployment(show_output = True)

In [ ]:
%%time
service.run('aachen_000000_000019_')

In [ ]:
%%time
import json
input_data = {"data": 'aachen_000000_000019_'}

service.run(json.dumps(input_data)['data'])
service.get_logs().split('\n')